Libraries:

In [ ]:
import lifesim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Range of delta in use:

In [ ]:
delta_ = np.insert(np.logspace(-10, -1, num=10, base=10), 0, 0, axis=0)
print(delta_)

Function for the analysis

In [ ]:
def det_yield(opt_scenario,   # time optimization scenario
            filter_bool,      # selection parameter
            delta, scenario): # range of delta, instrument scenario
    data = {}

    for val in delta:
        # import the previously saved catalog
        bus_read = lifesim.Bus()
        bus_read.data.options.set_scenario(scenario)
        
        # time optimization scenario
        if opt_scenario == 'S1':
            prefix = 'S1'
        elif opt_scenario == 'S2':
            prefix = ''
        else:
            print('ERROR: wrong optimization scenario specified')
            return None
        
        # instrument scenario
        if scenario == 'baseline':
            bus_read.data.import_catalog(input_path='C:/Users/kervy/Desktop/LIFEmission/LIFEsim/SavedData/'
                                         + prefix + f'DataBaselineTM_delta={val}.hdf5')
        elif scenario == 'optimistic':
            bus_read.data.import_catalog(input_path='C:/Users/kervy/Desktop/LIFEmission/LIFEsim/SavedData/'
                                         + prefix + f'DataOptimisticTM_delta={val}.hdf5')
        elif scenario == 'pessimistic':
            bus_read.data.import_catalog(input_path='C:/Users/kervy/Desktop/LIFEmission/LIFEsim/SavedData/'
                                         + prefix + f'DataPessimisticTM_delta={val}.hdf5')
        else:
            print('ERROR: wrong scenario specified')
            return None
    
        # retrieve the DataFrame object we will use
        df = bus_read.data.catalog
    
        # define the selection parameter
        if filter_bool == 'stype':
            param = df.stype
            bool_range = [param == 1, param == 2, param == 3, param == 4]
            filter_range = ["F", "G", "K", "M"]
        elif filter_bool == 'stypeHZ':
            param = df.stype
            bool_range = [(param == 1) & (df.habitable), (param == 2) & (df.habitable), 
                          (param == 3) & (df.habitable), (param == 4) & (df.habitable)]
            filter_range = ["F", "G", "K", "M"]
        elif filter_bool == 'distance':
            param = df.distance_s
            bool_range = [param < 3, (param >= 3) & (param < 5), 
                          (param >= 5) & (param < 7), (param >= 7) & (param < 9), 
                          (param >= 9) & (param < 11), (param >= 11) & (param < 13), 
                          (param >= 13) & (param < 15), param >= 15]
            filter_range = ["<3", "3-5", "5-7", "7-9", "9-11", "11-13", "13-15", ">15"]
        elif filter_bool == 'temperature':
            param = df.temp_p
            bool_range = [param < 125, (param >= 125) & (param < 150), (param >= 150) & (param < 175), 
                          (param >= 175) & (param < 200), (param >= 200) & (param < 225), 
                          (param >= 225) & (param < 250), (param >= 250) & (param < 275), 
                          (param >= 275) & (param < 300), param >= 300]
            filter_range = ["<125", "125-150", "150-175", "175-200", "200-225", "225-250", "250-275", "275-300", ">300"]
        elif filter_bool == 's_insolation':
            param = df.flux_p
            bool_range = [param < 0.2, (param >= 0.2) & (param < 0.4), (param >= 0.4) & (param < 0.6), 
                          (param >= 0.6) & (param < 0.8), (param >= 0.8) & (param < 1.0), 
                          (param >= 1.0) & (param < 1.2), (param >= 1.2) & (param < 1.4), 
                          (param >= 1.4) & (param < 1.6), param >= 1.6]
            filter_range = ["<0.2", "0.2-0.4", "0.4-0.6", "0.6-0.8", "0.8-1.0", "1.0-1.2", "1.2-1.4", "1.4-1.6", ">1.6"]
        elif filter_bool == 'angular_sep':
            param = df.angsep
            bool_range = [param < 0.1, (param >= 0.1) & (param < 0.2), 
                              (param >= 0.2) & (param < 0.3), param >= 0.3]
            filter_range = ["<0.1", "0.1-0.2", "0.2-0.3", ">0.3"]
        elif filter_bool == 'angular_sep2':
            param = df.angsep
            bool_range = [param < 0.1, (param >= 0.1) & (param < 0.3), param >= 0.3]
            filter_range = ["<0.1", "0.1-0.3", ">0.3"]
        else:
            print("ERROR: wrong selection parameter")
            return None
    
        # select the detectable planets according to the selection parameter
        nb_detected = []
        for filter_ in bool_range:
            selection = filter_ & df.detected
            nb_detected.append(selection.sum()/500)
            data[f'{val}'] = nb_detected

    ObsYield = pd.DataFrame(data, index=filter_range)
    return ObsYield

In [ ]:
def cat_yield(opt_scenario,   # time optimization scenario
            delta, scenario): # range of delta, instrument scenario

    # import the previously saved catalog
    bus_read = lifesim.Bus()
    bus_read.data.options.set_scenario(scenario)
        
    # time optimization scenario
    if opt_scenario == 'S1':
        prefix = 'S1'
    elif opt_scenario == 'S2':
        prefix = ''
    else:
        print('ERROR: wrong optimization scenario specified')
        return None
        
    # instrument scenario
    if scenario == 'baseline':
        bus_read.data.import_catalog(input_path='C:/Users/kervy/Desktop/LIFEmission/LIFEsim/SavedData/'
                                     + prefix + f'DataBaselineTM_delta={delta}.hdf5')
    elif scenario == 'optimistic':
        bus_read.data.import_catalog(input_path='C:/Users/kervy/Desktop/LIFEmission/LIFEsim/SavedData/'
                                     + prefix + f'DataOptimisticTM_delta={delta}.hdf5')
    elif scenario == 'pessimistic':
        bus_read.data.import_catalog(input_path='C:/Users/kervy/Desktop/LIFEmission/LIFEsim/SavedData/'
                                     + prefix + f'DataPessimisticTM_delta={delta}.hdf5')
    else:
        print('ERROR: wrong scenario specified')
        return None
    
    # retrieve the DataFrame object we will use
    df = bus_read.data.catalog

    return df

In [ ]:
def format_stype(stype):
    if stype == 1:
        return 'F'
    elif stype == 2:
        return 'G'
    elif stype == 3:
        return 'K'
    else:
        return 'M'
    
def computeflux(luminosity, distance):
    # the luminosity of the star is given in Sun luminosities
    Sun_lum = 3.828e+26 # [W]
    
    # the distance is given in parsec  
    parsec = 3.086e+16 # [m]
    
    # Earth's luminosity
    Earth_lum = 1.730e+17 # [W]
    
    # flux of if the Earth was at this distance
    flux_E = Earth_lum / (4 * np.pi * (distance * parsec)**2) # [Wm^-2]
    
    # result returned in units of Earth flux
    return ( luminosity * Sun_lum / (4 * np.pi * (distance * parsec)**2) ) / flux_E

def retrieve_all(opt_scenario, delta, inst_scenario):
    # yields the concatenated dataframe for all the simulations with a given
    # instrument and optimization scenario
    datalist = []
    for val in delta:
        sim_outcome = cat_yield(opt_scenario, val, inst_scenario)
        sim_outcome['delta'] = val
        sim_outcome['integration scenario'] = opt_scenario
        sim_outcome['instrument'] = inst_scenario
        sim_outcome['stellar type'] = sim_outcome['stype'].apply(format_stype)
        sim_outcome['ratioT'] = sim_outcome['temp_p']/sim_outcome['temp_s']
        sim_outcome['flux_s'] = computeflux(sim_outcome['l_sun'], sim_outcome['distance_s'])
        sim_outcome['contrast'] = sim_outcome['flux_p'] / sim_outcome['flux_s']
        datalist.append(sim_outcome)

    df = pd.concat(datalist)
    return df

Parameters for the graphs:

In [ ]:
# font
font = {'weight' : 'normal',
        'size'   : 12}

plt.rc('font', **font)

# path to save figures
path = 'C:/Users/kervy/Desktop/LIFEmission/LIFEsim/SavedFigures/'

Plot showing the modified transmission map:

In [ ]:
delta = 0.2
x = np.linspace(-4, 4, 200)

def T(x):
    return np.sin(x)**2

def T_tilde(x):
    return np.sin(x)**2 * (1-delta) + delta

def T_hat(x):
    return np.sin(x)**2 * (1-delta*np.exp(-delta*x**2)) + delta*np.exp(-delta*x**2)

fig = plt.figure()
plt.plot(x, T(x), 'b')
plt.plot(x, T_tilde(x), 'r--')
# plt.plot(x, T_hat(x), 'r--')

leg = ['$\sin^2(x)$', '$\sin^2(x)(1-\delta) + \delta$' ] #, '$\sin^2(x)(1- \delta e^{-\delta x^2}) + \delta e^{-\delta x^2}$']
plt.legend(leg, loc='upper right', ncol=1)

plt.xlabel('x')
plt.ylim(0,1)
plt.xlim(-4,4)
fig.set_size_inches(12, 4)
plt.savefig(path + 'TM_illustrative_plot', dpi=300)
plt.show()

# Scenario 1

### Baseline scenario, show same results for planets in HZ or not

In [ ]:
#retrieve both results
yield_HZ = det_yield('S1', 'stypeHZ', delta_, 'baseline')
yield_tot = det_yield('S1', 'stype', delta_, 'baseline')

In [ ]:
fig, ax = plt.subplots()
fig.figsize=(9, 3)

for stype in list(yield_tot.index.values):
    # attribute color to each stellar type
    if stype == 'F':
        clr = 'tomato'
    elif stype == 'G':
        clr = 'limegreen'
    elif stype == 'K':
        clr = 'orange'
    else: # stype == M
        clr = 'cornflowerblue'
    ax.plot(delta_, (yield_tot.loc[stype] - yield_tot.loc[stype][0])/yield_tot.loc[stype][0] * 100, '-', color=clr)

for stype in list(yield_HZ.index.values):
    # attribute color to each stellar type
    if stype == 'F':
        clr = 'tomato'
    elif stype == 'G':
        clr = 'limegreen'
    elif stype == 'K':
        clr = 'orange'
    else: # stype == M
        clr = 'cornflowerblue'
    ax.plot(delta_, (yield_HZ.loc[stype] - yield_HZ.loc[stype][0])/yield_HZ.loc[stype][0] * 100, '--', color=clr)
    
ax.set(xlabel='$\delta$', ylabel='$R(\delta)$ [%]')
ax.legend(list(yield_HZ.index.values), loc='best', ncol=1, title='stellar type')
ax.set_xscale('log')
ax.grid()
plt.xlim(0,0.1)

plt.savefig(path + 'S1_stypeHZvsTot', dpi=300, bbox_inches = 'tight')
plt.show()

### Bar plot HZ baseline, opt & pess.

In [ ]:
yB = det_yield('S1', 'stypeHZ', delta_, 'baseline')
yO = det_yield('S1', 'stypeHZ', delta_, 'optimistic')
yP = det_yield('S1', 'stypeHZ', delta_, 'pessimistic')

In [ ]:
# make the plot

fig, (ax, ax1) = plt.subplots(2, 1, sharex=True)
# fig, (ax, ax1) = plt.subplots(2, 1, figsize=(9, 9), sharex=True, gridspec_kw={'height_ratios': [1, 1]})

labels = []
fsize = 12

# restricted range since nothing happens for delta < 1.e-5
delta_loc = np.insert(np.logspace(-4, -1, num=4, base=10), 0, 0, axis=0)
for val in delta_loc:
    labels.append(f'{val}')

# set width of bars
barWidth = 0.25
    
# set position of the bars
r1 = np.arange(len(labels))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]

# set heights of bars
for stype in ['M','K','G','F']:
    bars1 = yO.loc[stype][labels]  #optimistic
    bars2 = yB.loc[stype][labels]  #baseline
    bars3 = yP.loc[stype][labels]  #pessimistic
 
    # Make the plot
    if stype == 'F':
        clr = 'tomato'
        ax.bar(r1, bars1, color=clr, width=barWidth, edgecolor='white', label='F')
        ax.bar(r2, bars2, color=clr, width=barWidth, edgecolor='white')
        ax.bar(r3, bars3, color=clr, width=barWidth, edgecolor='white')
    elif stype == 'G': # if not the first, stack on the previous series
        clr = 'limegreen'
        ax.bar(r1, bars1, color=clr, width=barWidth, edgecolor='white', label='G')
        ax.bar(r2, bars2, color=clr, width=barWidth, edgecolor='white')
        ax.bar(r3, bars3, color=clr, width=barWidth, edgecolor='white')
    elif stype == 'K': # if not the first, stack on the previous series
        clr = 'orange'
        ax.bar(r1, bars1, color=clr, width=barWidth, edgecolor='white', label='K')
        ax.bar(r2, bars2, color=clr, width=barWidth, edgecolor='white')
        ax.bar(r3, bars3, color=clr, width=barWidth, edgecolor='white')
    else: # if not the first, stack on the previous series
        clr = 'cornflowerblue'
        ax.bar(r1, bars1, color=clr, width=barWidth, edgecolor='white', label='M')
        ax.bar(r2, bars2, color=clr, width=barWidth, edgecolor='white')
        ax.bar(r3, bars3, color=clr, width=barWidth, edgecolor='white')

# add x ticks to the middle of the group bar
ax.set_xlabel('$\delta$', fontsize=fsize)
ax.set_ylabel('$N_{HZ}(\delta)$', fontsize=fsize)
plt.xticks([r + barWidth for r in range(len(bars1))], labels, fontsize=fsize)

# Create legend & Show graphic
ax.legend(loc='upper right', title='stellar type', ncol=2, fontsize=fsize)

# SECOND PLOT
# set heights of bars
for stype in ['M','K','G','F']:
    bars1 = yO.loc[stype][labels]  #optimistic
    bars2 = yB.loc[stype][labels]  #baseline
    bars3 = yP.loc[stype][labels]  #pessimistic
 
    # Make the plot
    if stype == 'F':
        clr = 'tomato'
        ax1.bar(r1, bars1, color=clr, width=barWidth, edgecolor='white', label='F')
        ax1.bar(r2, bars2, color=clr, width=barWidth, edgecolor='white')
        ax1.bar(r3, bars3, color=clr, width=barWidth, edgecolor='white')
    elif stype == 'G': # if not the first, stack on the previous series
        clr = 'limegreen'
        ax1.bar(r1, bars1, color=clr, width=barWidth, edgecolor='white', label='G')
        ax1.bar(r2, bars2, color=clr, width=barWidth, edgecolor='white')
        ax1.bar(r3, bars3, color=clr, width=barWidth, edgecolor='white')
    elif stype == 'K': # if not the first, stack on the previous series
        clr = 'orange'
        ax1.bar(r1, bars1, color=clr, width=barWidth, edgecolor='white', label='K')
        ax1.bar(r2, bars2, color=clr, width=barWidth, edgecolor='white')
        ax1.bar(r3, bars3, color=clr, width=barWidth, edgecolor='white')
    else: # if not the first, stack on the previous series
        clr = 'cornflowerblue'
        ax1.bar(r1, bars1, color=clr, width=barWidth, edgecolor='white', label='M')
        ax1.bar(r2, bars2, color=clr, width=barWidth, edgecolor='white')
        ax1.bar(r3, bars3, color=clr, width=barWidth, edgecolor='white')

# add x ticks to the middle of the group bar
ax1.set_xlabel('$\delta$', fontsize=fsize)
ax1.set_ylabel('$N_{HZ}(\delta)$', fontsize=fsize)
plt.xticks([r + barWidth for r in range(len(bars1))], labels, fontsize=fsize)
ax1.set_yscale('log')

fig.set_size_inches(6, 9)

plt.savefig(path + 'S1_stypeHZ_barplot', dpi=300, bbox_inches = 'tight')
plt.show()

### Temperature, baseline case

In [ ]:
tB = det_yield('S1', 'temperature', delta_, 'baseline')

In [ ]:
fig, ax = plt.subplots()
fig.figsize=(9, 3)

cm = plt.get_cmap('autumn')
NUM_COLORS = len(list(tB.index.values))
ax.set_prop_cycle(color=[cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)])

for temp in list(tB.index.values):
    ax.plot(delta_, (tB.loc[temp] - tB.loc[temp][0])/tB.loc[temp][0] * 100, '-')
    
ax.set(xlabel='$\delta$', ylabel='$R(\delta)$ [%]')
ax.legend(list(tB.index.values), loc='best', ncol=2, title='temperature [K]')
ax.set_xscale('log')
ax.grid()
plt.xlim(1.e-6,0.1)
#fig.set_size_inches(12, 4)

plt.savefig(path + 'S1_temp_baseline', dpi=300, bbox_inches = 'tight')
plt.show()

### Distance baseline

In [ ]:
dB = det_yield('S1', 'distance', delta_, 'baseline')

In [ ]:
fig, ax = plt.subplots()
fig.figsize=(9, 3)

cm = plt.get_cmap('plasma')
NUM_COLORS = len(list(tB.index.values))
ax.set_prop_cycle(color=[cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)])

for temp in list(dB.index.values):
    ax.plot(delta_, (dB.loc[temp] - dB.loc[temp][0])/dB.loc[temp][0] * 100, '-')
    
ax.set(xlabel='$\delta$', ylabel='$R(\delta)$ [%]')
ax.legend(list(dB.index.values), loc='best', ncol=2, title='distance [pc]')
ax.set_xscale('log')
ax.grid()
plt.xlim(1.e-6,0.1)

plt.savefig(path + 'S1_dist_baseline', dpi=300, bbox_inches = 'tight')
plt.show()

### Angular separation

In [ ]:
#retrieve the three results
aO = det_yield('S1', 'angular_sep2', delta_, 'optimistic')
aB = det_yield('S1', 'angular_sep2', delta_, 'baseline')
aP = det_yield('S1', 'angular_sep2', delta_, 'pessimistic')

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, sharey=True)
#fig.figsize=(9, 3)

for temp in list(aO.index.values):
    # attribute color to each stellar type
    if temp == "<0.1": 
        clr = 'tomato'
    elif temp == "0.1-0.3": 
        clr = 'limegreen'
    else: # angsep ">0.3"
        clr = 'cornflowerblue'
    ax1.plot(delta_, (aO.loc[temp] - aO.loc[temp][0])/aO.loc[temp][0] * 100, linestyle='solid', color=clr)

for temp in list(aB.index.values):
    # attribute color to each stellar type
    if temp == "<0.1": 
        clr = 'tomato'
    elif temp == "0.1-0.3": 
        clr = 'limegreen'
    else: # angsep ">0.3"
        clr = 'cornflowerblue'
    ax2.plot(delta_, (aB.loc[temp] - aB.loc[temp][0])/aB.loc[temp][0] * 100, linestyle='solid', color=clr)
    
for temp in list(aP.index.values):
    # attribute color to each stellar type
    if temp == "<0.1": 
        clr = 'tomato'
    elif temp == "0.1-0.3": 
        clr = 'limegreen'
    else: # angsep ">0.3"
        clr = 'cornflowerblue'
    ax3.plot(delta_, (aP.loc[temp] - aP.loc[temp][0])/aP.loc[temp][0] * 100, linestyle='solid', color=clr)
    
ax1.set(xlabel='$\delta$', ylabel='$R(\delta)$ [%]')
ax2.set(xlabel='$\delta$', ylabel='$R(\delta)$ [%]')
ax3.set(xlabel='$\delta$', ylabel='$R(\delta)$ [%]')

ax1.legend(list(aO.index.values), loc='best', ncol=2, title='angular separation [arcsec]')
ax2.legend(list(aO.index.values), loc='best', ncol=2, title='angular separation [arcsec]')
ax3.legend(list(aO.index.values), loc='best', ncol=2, title='angular separation [arcsec]')

ax1.title.set_text('optimistic')
ax2.title.set_text('baseline')
ax3.title.set_text('pessimistic')

ax1.set_xscale('log')

ax1.grid()
ax2.grid()
ax3.grid()

plt.xlim(1.e-6,0.1)
fig.set_size_inches(6, 12)

plt.savefig(path + 'S1_angsep', dpi=300, bbox_inches = 'tight')
plt.show()


### Comparative plot

In [ ]:
df1 = retrieve_all('S1', [0.0, 1.e-3, 1.e-2, 1.e-1], 'baseline')
dfHZ1 = df1[df1['detected'] & df1['habitable']]

df2 = retrieve_all('S2', [0.0, 1.e-3, 1.e-2, 1.e-1], 'baseline')
dfHZ2 = df2[df2['detected'] & df2['habitable']]

dfHZ = pd.concat([dfHZ1, dfHZ2])

In [ ]:
dfHZ[['flux_p', 'flux_s']]

In [ ]:
fs = 14 # fontsize
sns.set_theme(style="whitegrid")

# Plot 
graph = sns.relplot(x="angsep", y="ratioT", hue="stellar type", row='delta', col='integration scenario',
            sizes=(40, 400), alpha=.5, palette="muted",
            height=3.5, aspect = 1.7, data=dfHZ, legend='auto')

(graph.set_axis_labels("angular separation [arcsec]", "$T_p / T_s$", fontsize=fs)
      .set_titles("scenario {col_name}, $\delta =${row_name}", fontsize=fs)
      .tight_layout(w_pad=0))

graph.savefig(path +"AngsepRatioT.png",  dpi=300, bbox_inches = 'tight')

In [ ]:
fs = 14 # fontsize
sns.set_theme(style="whitegrid")

# Plot 
graph = sns.relplot(x="angsep", y="contrast", hue="stellar type", row='delta', col='integration scenario',
            sizes=(40, 400), alpha=.5, palette="muted",
            height=3, aspect = 1.7, data=dfHZ, legend='auto')

(graph.set_axis_labels("angular separation [arcsec]", "$F_p / F_s$", fontsize=fs)
      .set_titles("scenario {col_name}, $\delta =${row_name}", fontsize=fs)
      .tight_layout(w_pad=0))

graph.savefig(path +"AngsepContrast.png",  dpi=300, bbox_inches = 'tight')

In [ ]:
fs = 14 # fontsize
sns.set_theme(style="whitegrid")

# Plot 
graph = sns.relplot(x="angsep", y="contrast", hue="stellar type", row='delta', col='integration scenario',
            sizes=(40, 400), alpha=.5, palette="muted",
            height=3, aspect = 1.7, data=dfHZ, legend='auto')

(graph.set_axis_labels("angular separation [arcsec]", "$F_p / F_s$", fontsize=fs)
      .set_titles("scenario {col_name}, $\delta =${row_name}", fontsize=fs)
      .tight_layout(w_pad=0))

graph.set(xscale="log")
graph.set(yscale="log")

graph.savefig(path +"AngsepContrastLOG.png",  dpi=300, bbox_inches = 'tight')

In [ ]:
fs = 14 # fontsize
df = retrieve_all('S1', [0.0, 1.e-3], 'baseline')
temp = df[df['detected'] & df['habitable']]

# Plot 
sns.set_theme(style="whitegrid")

graph = sns.relplot(x="distance_s", y="angsep", hue="temp_p", size='radius_p',
            sizes=(20, 200), alpha=.5, palette="autumn", row='stellar type', col='delta',
            height=3, aspect=1.7, data=temp, legend='auto')

(graph.set_axis_labels("distance to the instrument [pc]", "angular separation [arcsec]", fontsize=fs)
      .set_titles("{row_name}-type star, $\delta =${col_name}", fontsize=fs)
      .tight_layout(w_pad=0))

graph.savefig(path +"S1outputHZ.png",  dpi=300, bbox_inches = 'tight')

In [ ]:
fs = 14 # fontsize
df = retrieve_all('S2', [0.0, 1.e-3], 'baseline')
temp = df[df['detected'] & df['habitable']]

# Plot 
sns.set_theme(style="whitegrid")

graph = sns.relplot(x="distance_s", y="angsep", hue="temp_p", size='radius_p',
            sizes=(20, 200), alpha=.5, palette="autumn", row='stellar type', col='delta',
            height=3, aspect=1.7, data=temp, legend='auto')

(graph.set_axis_labels("distance to the instrument [pc]", "angular separation [arcsec]", fontsize=fs)
      .set_titles("{row_name}-type star, $\delta =${col_name}", fontsize=fs)
      .tight_layout(w_pad=0))

graph.savefig(path +"S2outputHZ.png",  dpi=300, bbox_inches = 'tight')

In [ ]:
df1O = retrieve_all('S1', [0.0], 'optimistic')
df1B = retrieve_all('S1', [0.0], 'baseline')
df1P = retrieve_all('S1', [0.0], 'pessimistic')

df2O = retrieve_all('S2', [0.0], 'optimistic')
df2B = retrieve_all('S2', [0.0], 'baseline')
df2P = retrieve_all('S2', [0.0], 'pessimistic')

df = pd.concat([df1O, df1B, df1P, df2O, df2B, df2P])
temp = df[df['detected'] & df['habitable']]

In [ ]:
fs = 14 # fontsize

# Plot 
sns.set_theme(style="whitegrid")

graph = sns.relplot(x="distance_s", y="angsep", hue="temp_p",
            sizes=(20, 200), alpha=.5, palette="autumn", row='instrument', col='integration scenario',
            height=3, aspect=1.7, data=temp, legend='auto')

(graph.set_axis_labels("distance to the instrument [pc]", "angular separation [arcsec]", fontsize=fs)
      .set_titles("{row_name} configuration, integration scenario {col_name}", fontsize=fs)
      .tight_layout(w_pad=0))

graph.savefig(path +"outputInstrumentConfig.png",  dpi=300, bbox_inches = 'tight')

# Scenario 2

### Detected HZ vs overall

In [ ]:
yield_HZ = det_yield('S2', 'stypeHZ', delta_, 'baseline')
yield_tot = det_yield('S2', 'stype', delta_, 'baseline')

In [ ]:
#retrieve both results
fig, ax = plt.subplots()
fig.figsize=(9, 3)

for stype in list(yield_tot.index.values):
    # attribute color to each stellar type
    if stype == 'F':
        clr = 'tomato'
    elif stype == 'G':
        clr = 'limegreen'
    elif stype == 'K':
        clr = 'orange'
    else: # stype == M
        clr = 'cornflowerblue'
    ax.plot(delta_, (yield_tot.loc[stype] - yield_tot.loc[stype][0])/yield_tot.loc[stype][0] * 100, '-', color=clr)

for stype in list(yield_HZ.index.values):
    # attribute color to each stellar type
    if stype == 'F':
        clr = 'tomato'
    elif stype == 'G':
        clr = 'limegreen'
    elif stype == 'K':
        clr = 'orange'
    else: # stype == M
        clr = 'cornflowerblue'
    ax.plot(delta_, (yield_HZ.loc[stype] - yield_HZ.loc[stype][0])/yield_HZ.loc[stype][0] * 100, '--', color=clr)
    
ax.set(xlabel='$\delta$', ylabel='$R(\delta)$ [%]')
ax.legend(list(yield_HZ.index.values), loc='best', ncol=1, title='stellar type')
ax.set_xscale('log')
ax.grid()
plt.xlim(0,0.1)

plt.savefig(path + 'S2_stypeHZvsTot', dpi=300, bbox_inches = 'tight')
plt.show()

### Bar plot

In [ ]:
yB = det_yield('S2', 'stypeHZ', delta_, 'baseline')
yO = det_yield('S2', 'stypeHZ', delta_, 'optimistic')
yP = det_yield('S2', 'stypeHZ', delta_, 'pessimistic')

In [ ]:
# make the plot
fig, (ax, ax1) = plt.subplots(2, 1,sharex=True)

labels = []
fsize = 12

# restricted range since nothing happens for delta < 1.e-5
delta_loc = np.insert(np.logspace(-4, -1, num=4, base=10), 0, 0, axis=0)
for val in delta_loc:
    labels.append(f'{val}')

# set width of bars
barWidth = 0.25
    
# set position of the bars
r1 = np.arange(len(labels))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]

# set heights of bars
for stype in ['M','K','G','F']:
    bars1 = yO.loc[stype][labels]  #optimistic
    bars2 = yB.loc[stype][labels]  #baseline
    bars3 = yP.loc[stype][labels]  #pessimistic
 
    # Make the plot
    if stype == 'F':
        clr = 'tomato'
        ax.bar(r1, bars1, color=clr, width=barWidth, edgecolor='white', label='F')
        ax.bar(r2, bars2, color=clr, width=barWidth, edgecolor='white')
        ax.bar(r3, bars3, color=clr, width=barWidth, edgecolor='white')
    elif stype == 'G': # if not the first, stack on the previous series
        clr = 'limegreen'
        ax.bar(r1, bars1, color=clr, width=barWidth, edgecolor='white', label='G')
        ax.bar(r2, bars2, color=clr, width=barWidth, edgecolor='white')
        ax.bar(r3, bars3, color=clr, width=barWidth, edgecolor='white')
    elif stype == 'K': # if not the first, stack on the previous series
        clr = 'orange'
        ax.bar(r1, bars1, color=clr, width=barWidth, edgecolor='white', label='K')
        ax.bar(r2, bars2, color=clr, width=barWidth, edgecolor='white')
        ax.bar(r3, bars3, color=clr, width=barWidth, edgecolor='white')
    else: # if not the first, stack on the previous series
        clr = 'cornflowerblue'
        ax.bar(r1, bars1, color=clr, width=barWidth, edgecolor='white', label='M')
        ax.bar(r2, bars2, color=clr, width=barWidth, edgecolor='white')
        ax.bar(r3, bars3, color=clr, width=barWidth, edgecolor='white')

# add x ticks to the middle of the group bar
ax.set_xlabel('$\delta$', fontsize=fsize)
ax.set_ylabel('$N_{HZ}(\delta)$', fontsize=fsize)
plt.xticks([r + barWidth for r in range(len(bars1))], labels, fontsize=fsize)

# Create legend & Show graphic
ax.legend(loc='upper right', title='stellar type', ncol=2, fontsize=fsize)

# SECOND PLOT
# set heights of bars
for stype in ['M','K','G','F']:
    bars1 = yO.loc[stype][labels]  #optimistic
    bars2 = yB.loc[stype][labels]  #baseline
    bars3 = yP.loc[stype][labels]  #pessimistic
 
    # Make the plot
    if stype == 'F':
        clr = 'tomato'
        ax1.bar(r1, bars1, color=clr, width=barWidth, edgecolor='white', label='F')
        ax1.bar(r2, bars2, color=clr, width=barWidth, edgecolor='white')
        ax1.bar(r3, bars3, color=clr, width=barWidth, edgecolor='white')
    elif stype == 'G': # if not the first, stack on the previous series
        clr = 'limegreen'
        ax1.bar(r1, bars1, color=clr, width=barWidth, edgecolor='white', label='G')
        ax1.bar(r2, bars2, color=clr, width=barWidth, edgecolor='white')
        ax1.bar(r3, bars3, color=clr, width=barWidth, edgecolor='white')
    elif stype == 'K': # if not the first, stack on the previous series
        clr = 'orange'
        ax1.bar(r1, bars1, color=clr, width=barWidth, edgecolor='white', label='K')
        ax1.bar(r2, bars2, color=clr, width=barWidth, edgecolor='white')
        ax1.bar(r3, bars3, color=clr, width=barWidth, edgecolor='white')
    else: # if not the first, stack on the previous series
        clr = 'cornflowerblue'
        ax1.bar(r1, bars1, color=clr, width=barWidth, edgecolor='white', label='M')
        ax1.bar(r2, bars2, color=clr, width=barWidth, edgecolor='white')
        ax1.bar(r3, bars3, color=clr, width=barWidth, edgecolor='white')

fig.set_size_inches(6, 9)
    
# add x ticks to the middle of the group bar
ax1.set_xlabel('$\delta$', fontsize=fsize)
ax1.set_ylabel('$N_{HZ}(\delta)$', fontsize=fsize)
plt.xticks([r + barWidth for r in range(len(bars1))], labels, fontsize=fsize)
ax1.set_yscale('log')

plt.savefig(path + 'S2_stypeHZ_barplot', dpi=300, bbox_inches = 'tight')
plt.show()

### Temperature

In [ ]:
tB = det_yield('S2', 'temperature', delta_, 'baseline')

In [ ]:
fig, ax = plt.subplots()
fig.figsize=(9, 3)

cm = plt.get_cmap('autumn')
NUM_COLORS = len(list(tB.index.values))
ax.set_prop_cycle(color=[cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)])

for temp in list(tB.index.values):
    ax.plot(delta_, (tB.loc[temp] - tB.loc[temp][0])/tB.loc[temp][0] * 100, '-')
    
ax.set(xlabel='$\delta$', ylabel='$R(\delta)$ [%]')
ax.legend(list(tB.index.values), loc='best', ncol=2, title='temperature [K]')
ax.set_xscale('log')
ax.grid()
plt.xlim(1.e-6,0.1)
#fig.set_size_inches(12, 4)

plt.savefig(path + 'S2_temp_baseline', dpi=300, bbox_inches = 'tight')
plt.show()

### Distance

In [ ]:
dB = det_yield('S2', 'distance', delta_, 'baseline')

In [ ]:
fig, ax = plt.subplots()
fig.figsize=(9, 3)

cm = plt.get_cmap('plasma')
NUM_COLORS = len(list(tB.index.values))
ax.set_prop_cycle(color=[cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)])

for temp in list(dB.index.values):
    ax.plot(delta_, (dB.loc[temp] - dB.loc[temp][0])/dB.loc[temp][0] * 100, '-')
    
ax.set(xlabel='$\delta$', ylabel='$R(\delta)$ [%]')
ax.legend(list(dB.index.values), loc='best', ncol=2, title='distance [pc]')
ax.set_xscale('log')
ax.grid()
plt.xlim(1.e-6,0.1)

plt.savefig(path + 'S2_dist_baseline', dpi=300, bbox_inches = 'tight')
plt.show()

### Angular separation

In [ ]:
#retrieve the three results
aO = det_yield('S2', 'angular_sep2', delta_, 'optimistic')
aB = det_yield('S2', 'angular_sep2', delta_, 'baseline')
aP = det_yield('S2', 'angular_sep2', delta_, 'pessimistic')

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, sharey=True)
#fig.figsize=(9, 3)

for temp in list(aO.index.values):
    # attribute color to each stellar type
    if temp == "<0.1": 
        clr = 'tomato'
    elif temp == "0.1-0.3": 
        clr = 'limegreen'
    else: # angsep ">0.3"
        clr = 'cornflowerblue'
    ax1.plot(delta_, (aO.loc[temp] - aO.loc[temp][0])/aO.loc[temp][0] * 100, linestyle='solid', color=clr)

for temp in list(aB.index.values):
    # attribute color to each stellar type
    if temp == "<0.1": 
        clr = 'tomato'
    elif temp == "0.1-0.3": 
        clr = 'limegreen'
    else: # angsep ">0.3"
        clr = 'cornflowerblue'
    ax2.plot(delta_, (aB.loc[temp] - aB.loc[temp][0])/aB.loc[temp][0] * 100, linestyle='solid', color=clr)
    
for temp in list(aP.index.values):
    # attribute color to each stellar type
    if temp == "<0.1": 
        clr = 'tomato'
    elif temp == "0.1-0.3": 
        clr = 'limegreen'
    else: # angsep ">0.3"
        clr = 'cornflowerblue'
    ax3.plot(delta_, (aP.loc[temp] - aP.loc[temp][0])/aP.loc[temp][0] * 100, linestyle='solid', color=clr)
    
ax1.set(xlabel='$\delta$', ylabel='$R(\delta)$ [%]')
ax2.set(xlabel='$\delta$', ylabel='$R(\delta)$ [%]')
ax3.set(xlabel='$\delta$', ylabel='$R(\delta)$ [%]')

ax1.legend(list(aO.index.values), loc='best', ncol=2, title='angular separation [arcsec]')
ax2.legend(list(aO.index.values), loc='best', ncol=2, title='angular separation [arcsec]')
ax3.legend(list(aO.index.values), loc='best', ncol=2, title='angular separation [arcsec]')

ax1.title.set_text('optimistic')
ax2.title.set_text('baseline')
ax3.title.set_text('pessimistic')

ax1.set_xscale('log')

ax1.grid()
ax2.grid()
ax3.grid()

plt.xlim(1.e-6,0.1)
fig.set_size_inches(6, 12)

plt.savefig(path + 'S2_angsep', dpi=300, bbox_inches = 'tight')
plt.show()


# Additional Graphs    

In [ ]:
#retrieve both results
yield_HZ = det_yield('S1', 'stypeHZ', delta_, 'optimistic')
yield_tot = det_yield('S1', 'stype', delta_, 'optimistic')

fig, ax = plt.subplots()
fig.figsize=(9, 3)

for stype in list(yield_tot.index.values):
    # attribute color to each stellar type
    if stype == 'F':
        clr = 'tomato'
    elif stype == 'G':
        clr = 'limegreen'
    elif stype == 'K':
        clr = 'orange'
    else: # stype == M
        clr = 'cornflowerblue'
    ax.plot(delta_, (yield_tot.loc[stype] - yield_tot.loc[stype][0])/yield_tot.loc[stype][0] * 100, '-', color=clr)

for stype in list(yield_HZ.index.values):
    # attribute color to each stellar type
    if stype == 'F':
        clr = 'tomato'
    elif stype == 'G':
        clr = 'limegreen'
    elif stype == 'K':
        clr = 'orange'
    else: # stype == M
        clr = 'cornflowerblue'
    ax.plot(delta_, (yield_HZ.loc[stype] - yield_HZ.loc[stype][0])/yield_HZ.loc[stype][0] * 100, '--', color=clr)
    
ax.set(xlabel='$\delta$', ylabel='$R(\delta)$ [%]')
ax.legend(list(yield_HZ.index.values), loc='best', ncol=1, title='stellar type')
ax.set_xscale('log')
ax.grid()
plt.xlim(0,0.1)

plt.savefig(path + 'S1_stypeHZvsTot_Optimistic', dpi=300, bbox_inches = 'tight')
plt.show()

In [ ]:
#retrieve both results
yield_HZ = det_yield('S1', 'stypeHZ', delta_, 'pessimistic')
yield_tot = det_yield('S1', 'stype', delta_, 'pessimistic')

fig, ax = plt.subplots()
fig.figsize=(9, 3)

for stype in list(yield_tot.index.values):
    # attribute color to each stellar type
    if stype == 'F':
        clr = 'tomato'
    elif stype == 'G':
        clr = 'limegreen'
    elif stype == 'K':
        clr = 'orange'
    else: # stype == M
        clr = 'cornflowerblue'
    ax.plot(delta_, (yield_tot.loc[stype] - yield_tot.loc[stype][0])/yield_tot.loc[stype][0] * 100, '-', color=clr)

for stype in list(yield_HZ.index.values):
    # attribute color to each stellar type
    if stype == 'F':
        clr = 'tomato'
    elif stype == 'G':
        clr = 'limegreen'
    elif stype == 'K':
        clr = 'orange'
    else: # stype == M
        clr = 'cornflowerblue'
    ax.plot(delta_, (yield_HZ.loc[stype] - yield_HZ.loc[stype][0])/yield_HZ.loc[stype][0] * 100, '--', color=clr)
    
ax.set(xlabel='$\delta$', ylabel='$R(\delta)$ [%]')
ax.legend(list(yield_HZ.index.values), loc='best', ncol=1, title='stellar type')
ax.set_xscale('log')
ax.grid()
plt.xlim(0,0.1)

plt.savefig(path + 'S1_stypeHZvsTot_Pessimistic', dpi=300, bbox_inches = 'tight')
plt.show()

In [ ]:
#retrieve both results
yield_HZ = det_yield('S2', 'stypeHZ', delta_, 'optimistic')
yield_tot = det_yield('S2', 'stype', delta_, 'optimistic')

fig, ax = plt.subplots()
fig.figsize=(9, 3)

for stype in list(yield_tot.index.values):
    # attribute color to each stellar type
    if stype == 'F':
        clr = 'tomato'
    elif stype == 'G':
        clr = 'limegreen'
    elif stype == 'K':
        clr = 'orange'
    else: # stype == M
        clr = 'cornflowerblue'
    ax.plot(delta_, (yield_tot.loc[stype] - yield_tot.loc[stype][0])/yield_tot.loc[stype][0] * 100, '-', color=clr)

for stype in list(yield_HZ.index.values):
    # attribute color to each stellar type
    if stype == 'F':
        clr = 'tomato'
    elif stype == 'G':
        clr = 'limegreen'
    elif stype == 'K':
        clr = 'orange'
    else: # stype == M
        clr = 'cornflowerblue'
    ax.plot(delta_, (yield_HZ.loc[stype] - yield_HZ.loc[stype][0])/yield_HZ.loc[stype][0] * 100, '--', color=clr)
    
ax.set(xlabel='$\delta$', ylabel='$R(\delta)$ [%]')
ax.legend(list(yield_HZ.index.values), loc='best', ncol=1, title='stellar type')
ax.set_xscale('log')
ax.grid()
plt.xlim(0,0.1)

plt.savefig(path + 'S2_stypeHZvsTot_Optimistic', dpi=300, bbox_inches = 'tight')
plt.show()

In [ ]:
#retrieve both results
yield_HZ = det_yield('S2', 'stypeHZ', delta_, 'pessimistic')
yield_tot = det_yield('S2', 'stype', delta_, 'pessimistic')

fig, ax = plt.subplots()
fig.figsize=(9, 3)

for stype in list(yield_tot.index.values):
    # attribute color to each stellar type
    if stype == 'F':
        clr = 'tomato'
    elif stype == 'G':
        clr = 'limegreen'
    elif stype == 'K':
        clr = 'orange'
    else: # stype == M
        clr = 'cornflowerblue'
    ax.plot(delta_, (yield_tot.loc[stype] - yield_tot.loc[stype][0])/yield_tot.loc[stype][0] * 100, '-', color=clr)

for stype in list(yield_HZ.index.values):
    # attribute color to each stellar type
    if stype == 'F':
        clr = 'tomato'
    elif stype == 'G':
        clr = 'limegreen'
    elif stype == 'K':
        clr = 'orange'
    else: # stype == M
        clr = 'cornflowerblue'
    ax.plot(delta_, (yield_HZ.loc[stype] - yield_HZ.loc[stype][0])/yield_HZ.loc[stype][0] * 100, '--', color=clr)
    
ax.set(xlabel='$\delta$', ylabel='$R(\delta)$ [%]')
ax.legend(list(yield_HZ.index.values), loc='best', ncol=1, title='stellar type')
ax.set_xscale('log')
ax.grid()
plt.xlim(0,0.1)

plt.savefig(path + 'S2_stypeHZvsTot_Pessimistic', dpi=300, bbox_inches = 'tight')
plt.show()

In [ ]:
tB = det_yield('S1', 'temperature', delta_, 'optimistic')

fig, ax = plt.subplots()
fig.figsize=(9, 3)

cm = plt.get_cmap('autumn')
NUM_COLORS = len(list(tB.index.values))
ax.set_prop_cycle(color=[cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)])

for temp in list(tB.index.values):
    ax.plot(delta_, (tB.loc[temp] - tB.loc[temp][0])/tB.loc[temp][0] * 100, '-')
    
ax.set(xlabel='$\delta$', ylabel='$R(\delta)$ [%]')
ax.legend(list(tB.index.values), loc='best', ncol=2, title='temperature [K]')
ax.set_xscale('log')
ax.grid()
plt.xlim(1.e-6,0.1)
#fig.set_size_inches(12, 4)

plt.savefig(path + 'S1_temp_optimistic', dpi=300, bbox_inches = 'tight')
plt.show()

In [ ]:
tB = det_yield('S1', 'temperature', delta_, 'pessimistic')

fig, ax = plt.subplots()
fig.figsize=(9, 3)

cm = plt.get_cmap('autumn')
NUM_COLORS = len(list(tB.index.values))
ax.set_prop_cycle(color=[cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)])

for temp in list(tB.index.values):
    ax.plot(delta_, (tB.loc[temp] - tB.loc[temp][0])/tB.loc[temp][0] * 100, '-')
    
ax.set(xlabel='$\delta$', ylabel='$R(\delta)$ [%]')
ax.legend(list(tB.index.values), loc='best', ncol=2, title='temperature [K]')
ax.set_xscale('log')
ax.grid()
plt.xlim(1.e-6,0.1)
#fig.set_size_inches(12, 4)

plt.savefig(path + 'S1_temp_pessimistic', dpi=300, bbox_inches = 'tight')
plt.show()

In [ ]:
tB = det_yield('S2', 'temperature', delta_, 'optimistic')

fig, ax = plt.subplots()
fig.figsize=(9, 3)

cm = plt.get_cmap('autumn')
NUM_COLORS = len(list(tB.index.values))
ax.set_prop_cycle(color=[cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)])

for temp in list(tB.index.values):
    ax.plot(delta_, (tB.loc[temp] - tB.loc[temp][0])/tB.loc[temp][0] * 100, '-')
    
ax.set(xlabel='$\delta$', ylabel='$R(\delta)$ [%]')
ax.legend(list(tB.index.values), loc='best', ncol=2, title='temperature [K]')
ax.set_xscale('log')
ax.grid()
plt.xlim(1.e-6,0.1)
#fig.set_size_inches(12, 4)

plt.savefig(path + 'S2_temp_optimistic', dpi=300, bbox_inches = 'tight')
plt.show()

In [ ]:
tB = det_yield('S2', 'temperature', delta_, 'pessimistic')

fig, ax = plt.subplots()
fig.figsize=(9, 3)

cm = plt.get_cmap('autumn')
NUM_COLORS = len(list(tB.index.values))
ax.set_prop_cycle(color=[cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)])

for temp in list(tB.index.values):
    ax.plot(delta_, (tB.loc[temp] - tB.loc[temp][0])/tB.loc[temp][0] * 100, '-')
    
ax.set(xlabel='$\delta$', ylabel='$R(\delta)$ [%]')
ax.legend(list(tB.index.values), loc='best', ncol=2, title='temperature [K]')
ax.set_xscale('log')
ax.grid()
plt.xlim(1.e-6,0.1)
#fig.set_size_inches(12, 4)

plt.savefig(path + 'S2_temp_pessimistic', dpi=300, bbox_inches = 'tight')
plt.show()

In [ ]:
dB = det_yield('S1', 'distance', delta_, 'optimistic')

In [ ]:
fig, ax = plt.subplots()
fig.figsize=(9, 3)

cm = plt.get_cmap('plasma')
NUM_COLORS = len(list(tB.index.values))
ax.set_prop_cycle(color=[cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)])

for temp in list(dB.index.values):
    ax.plot(delta_, (dB.loc[temp] - dB.loc[temp][0])/dB.loc[temp][0] * 100, '-')
    
ax.set(xlabel='$\delta$', ylabel='$R(\delta)$ [%]')
ax.legend(list(dB.index.values), loc='best', ncol=2, title='distance [pc]')
ax.set_xscale('log')
ax.grid()
plt.xlim(1.e-6,0.1)

plt.savefig(path + 'S1_dist_optimistic', dpi=300, bbox_inches = 'tight')
plt.show()

In [ ]:
dB = det_yield('S1', 'distance', delta_, 'pessimistic')

In [ ]:
fig, ax = plt.subplots()
fig.figsize=(9, 3)

cm = plt.get_cmap('plasma')
NUM_COLORS = len(list(tB.index.values))
ax.set_prop_cycle(color=[cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)])

for temp in list(dB.index.values):
    ax.plot(delta_, (dB.loc[temp] - dB.loc[temp][0])/dB.loc[temp][0] * 100, '-')
    
ax.set(xlabel='$\delta$', ylabel='$R(\delta)$ [%]')
ax.legend(list(dB.index.values), loc='best', ncol=2, title='distance [pc]')
ax.set_xscale('log')
ax.grid()
plt.xlim(1.e-6,0.1)

plt.savefig(path + 'S1_dist_pessimistic', dpi=300, bbox_inches = 'tight')
plt.show()

In [ ]:
dB = det_yield('S2', 'distance', delta_, 'optimistic')

In [ ]:
fig, ax = plt.subplots()
fig.figsize=(9, 3)

cm = plt.get_cmap('plasma')
NUM_COLORS = len(list(tB.index.values))
ax.set_prop_cycle(color=[cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)])

for temp in list(dB.index.values):
    ax.plot(delta_, (dB.loc[temp] - dB.loc[temp][0])/dB.loc[temp][0] * 100, '-')
    
ax.set(xlabel='$\delta$', ylabel='$R(\delta)$ [%]')
ax.legend(list(dB.index.values), loc='best', ncol=2, title='distance [pc]')
ax.set_xscale('log')
ax.grid()
plt.xlim(1.e-6,0.1)

plt.savefig(path + 'S2_dist_optimistic', dpi=300, bbox_inches = 'tight')
plt.show()

In [ ]:
dB = det_yield('S2', 'distance', delta_, 'pessimistic')

In [ ]:
fig, ax = plt.subplots()
fig.figsize=(9, 3)

cm = plt.get_cmap('plasma')
NUM_COLORS = len(list(tB.index.values))
ax.set_prop_cycle(color=[cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)])

for temp in list(dB.index.values):
    ax.plot(delta_, (dB.loc[temp] - dB.loc[temp][0])/dB.loc[temp][0] * 100, '-')
    
ax.set(xlabel='$\delta$', ylabel='$R(\delta)$ [%]')
ax.legend(list(dB.index.values), loc='best', ncol=2, title='distance [pc]')
ax.set_xscale('log')
ax.grid()
plt.xlim(1.e-6,0.1)

plt.savefig(path + 'S2_dist_pessimistic', dpi=300, bbox_inches = 'tight')
plt.show()

# Discussion

In [ ]:
#retrieve both results, scenario 1
yHZ_1 = det_yield('S1', 'stypeHZ', delta_, 'baseline')
yTO_1 = det_yield('S1', 'stype', delta_, 'baseline')

#retrieve both results, scenario 2
yHZ_2 = det_yield('S2', 'stypeHZ', delta_, 'baseline')
yTO_2 = det_yield('S2', 'stype', delta_, 'baseline')

In [ ]:
fig, ax = plt.subplots()
fig.figsize=(9, 3)

for stype in list(yHZ_1.index.values):
    # attribute color to each stellar type
    if stype == 'F':
        clr = 'tomato'
    elif stype == 'G':
        clr = 'limegreen'
    elif stype == 'K':
        clr = 'orange'
    elif stype == 'M':
        clr = 'cornflowerblue'
    ax.plot(delta_, yHZ_1.loc[stype] / yTO_1.loc[stype], '-', color=clr)

for index in list(yHZ_2.index.values):
    # attribute color to each stellar type
    if index == 'F':
        clr = 'tomato'
    elif index == 'G':
        clr = 'limegreen'
    elif index == 'K':
        clr = 'orange'
    elif index == 'M':
        clr = 'cornflowerblue'
    ax.plot(delta_, yHZ_2.loc[index] / yTO_2.loc[index], '--', color=clr)
    
ax.set(xlabel='$\delta$', ylabel='$Q_N(\delta)$')
ax.legend(list(yHZ_1.index.values), loc='best', ncol=2, title='stellar type')
ax.set_xscale('log')
ax.grid()
plt.xlim(1.e-6,0.1)

plt.savefig(path + 'StypeHZvsTot_S1vsS2', dpi=300, bbox_inches = 'tight')
plt.show()

In [ ]:
df = retrieve_all('S1', [0.0, 1.e-3, 1.e-1], 'baseline')
df_det = df[df['detected']]

sns.set_theme(style="white")

# Plot 
sns.relplot(x="angsep", y="temp_p", hue="habitable", col='delta',
            sizes=(40, 400), alpha=.5, palette="muted",
            height=6, data=df_det, legend='auto')

## T/angsep for different stellar types

In [ ]:
def det_specific(opt_scenario,    # time optimization scenario
                filter_bool,      # selection parameter
                stype,             # stellar type
                delta, scenario): # range of delta, instrument scenario
    data = {}

    for val in delta:
        # import the previously saved catalog
        bus_read = lifesim.Bus()
        bus_read.data.options.set_scenario(scenario)
        
        # time optimization scenario
        if opt_scenario == 'S1':
            prefix = 'S1'
        elif opt_scenario == 'S2':
            prefix = ''
        else:
            print('ERROR: wrong optimization scenario specified')
            return None
        
        # instrument scenario
        if scenario == 'baseline':
            bus_read.data.import_catalog(input_path='C:/Users/kervy/Desktop/LIFEmission/LIFEsim/SavedData/'
                                         + prefix + f'DataBaselineTM_delta={val}.hdf5')
        elif scenario == 'optimistic':
            bus_read.data.import_catalog(input_path='C:/Users/kervy/Desktop/LIFEmission/LIFEsim/SavedData/'
                                         + prefix + f'DataOptimisticTM_delta={val}.hdf5')
        elif scenario == 'pessimistic':
            bus_read.data.import_catalog(input_path='C:/Users/kervy/Desktop/LIFEmission/LIFEsim/SavedData/'
                                         + prefix + f'DataPessimisticTM_delta={val}.hdf5')
        else:
            print('ERROR: wrong scenario specified')
            return None
    
        # retrieve the DataFrame object we will use
        df = bus_read.data.catalog
    
        # define the selection parameter
        if stype == 'F':
            param_star = 1
        elif stype == 'G':
            param_star = 2
        elif stype == 'K':
            param_star = 3
        elif stype == 'M':
            param_star = 4
        else:
            print("Wrong stellar type")

        if filter_bool == 'temperature':
            param = df.temp_p
            bool_range = [(df.stype == param_star) & (df.habitable) & (param < 125), 
                          (df.stype == param_star) & (df.habitable) & (param >= 125) & (param < 150), 
                          (df.stype == param_star) & (df.habitable) & (param >= 150) & (param < 175), 
                          (df.stype == param_star) & (df.habitable) & (param >= 175) & (param < 200), 
                          (df.stype == param_star) & (df.habitable) & (param >= 200) & (param < 225), 
                          (df.stype == param_star) & (df.habitable) & (param >= 225) & (param < 250), 
                          (df.stype == param_star) & (df.habitable) & (param >= 250) & (param < 275), 
                          (df.stype == param_star) & (df.habitable) & (param >= 275) & (param < 300), 
                          (df.stype == param_star) & (df.habitable) & (param >= 300)]
            filter_range = ["<125", "125-150", "150-175", "175-200", "200-225", "225-250", "250-275", "275-300", ">300"]
        elif filter_bool == 's_insolation':
            param = df.flux_p
            bool_range = [(df.stype == param_star) & (df.habitable) & (param < 0.2), 
                          (df.stype == param_star) & (df.habitable) & (param >= 0.2) & (param < 0.4), 
                          (df.stype == param_star) & (df.habitable) & (param >= 0.4) & (param < 0.6), 
                          (df.stype == param_star) & (df.habitable) & (param >= 0.6) & (param < 0.8),
                          (df.stype == param_star) & (df.habitable) & (param >= 0.8) & (param < 1.0), 
                          (df.stype == param_star) & (df.habitable) & (param >= 1.0) & (param < 1.2), 
                          (df.stype == param_star) & (df.habitable) & (param >= 1.2) & (param < 1.4), 
                          (df.stype == param_star) & (df.habitable) & (param >= 1.4) & (param < 1.6), 
                          (df.stype == param_star) & (df.habitable) & (param >= 1.6)]
            filter_range = ["<0.2", "0.2-0.4", "0.4-0.6", "0.6-0.8", "0.8-1.0", "1.0-1.2", "1.2-1.4", "1.4-1.6", ">1.6"]
        elif filter_bool == 'angular_sep':
            param = df.angsep
            bool_range = [(df.stype == param_star) & (df.habitable) & (param < 0.1), 
                          (df.stype == param_star) & (df.habitable) & (param >= 0.1) & (param < 0.2), 
                          (df.stype == param_star) & (df.habitable) & (param >= 0.2) & (param < 0.3), 
                          (df.stype == param_star) & (df.habitable) & (param >= 0.3)]
            filter_range = ["<0.1", "0.1-0.2", "0.2-0.3", ">0.3"]
        elif filter_bool == 'angular_sep2':
            param = df.angsep
            bool_range = [(df.stype == param_star) & (df.habitable) & (param < 0.1), 
                          (df.stype == param_star) & (df.habitable) & (param >= 0.1) & (param < 0.3), 
                          (df.stype == param_star) & (df.habitable) & (param >= 0.3)]
            filter_range = ["<0.1", "0.1-0.3", ">0.3"]
        else:
            print("ERROR: wrong selection parameter")
            return None
    
        # select the detectable planets according to the selection parameter
        nb_detected = []
        for filter_ in bool_range:
            selection = filter_ & df.detected
            nb_detected.append(selection.sum()/500)
            data[f'{val}'] = nb_detected

    ObsYield = pd.DataFrame(data, index=filter_range)
    return ObsYield

In [ ]:
def plot_temp(datalist, name):
    fig, ax = plt.subplots()
    fig.figsize=(9, 3)

    for temp in list(datalist.index.values):
        ax.plot(delta_, (datalist.loc[temp] - datalist.loc[temp][0])/datalist.loc[temp][0] * 100, '-')
    
    ax.set(xlabel='$\delta$', ylabel='$R(\delta)$ [%]')
    ax.legend(list(yieldM.index.values), loc='best', ncol=1, title='temperature [K]')
    ax.set_xscale('log')
    ax.grid()
    plt.xlim(1.e-6,0.1)
    #fig.set_size_inches(12, 4)

    plt.savefig(path + name, dpi=300, bbox_inches = 'tight')
    plt.show()

In [ ]:
yieldM = det_specific('S2', 'temperature', 'M', delta_, 'baseline')
yieldK = det_specific('S2', 'temperature', 'K', delta_, 'baseline')
yieldG = det_specific('S2', 'temperature', 'G', delta_, 'baseline')
yieldF = det_specific('S2', 'temperature', 'F', delta_, 'baseline')

In [ ]:
plot_temp(yieldM, 'S2_HZtemp_M')
plot_temp(yieldK, 'S2_HZtemp_K')
plot_temp(yieldG, 'S2_HZtemp_G')
plot_temp(yieldF, 'S2_HZtemp_F')

In [ ]:
def plot_angsep(datalist, name):
    fig, ax = plt.subplots()
    fig.figsize=(9, 3)

    for angle in list(datalist.index.values):
        ax.plot(delta_, (datalist.loc[angle] - datalist.loc[angle][0])/datalist.loc[angle][0] * 100, '-')
    
    ax.set(xlabel='$\delta$', ylabel='$R(\delta)$ [%]')
    ax.legend(list(yieldM.index.values), loc='best', ncol=2, title='angular separation [arcsec]')
    ax.set_xscale('log')
    ax.grid()
    plt.xlim(1.e-6,0.1)
    #fig.set_size_inches(12, 4)

    plt.savefig(path + name, dpi=300, bbox_inches = 'tight')
    plt.show()

In [ ]:
yieldM = det_specific('S2', 'angular_sep2', 'M', delta_, 'baseline')
yieldK = det_specific('S2', 'angular_sep2', 'K', delta_, 'baseline')
yieldG = det_specific('S2', 'angular_sep2', 'G', delta_, 'baseline')
yieldF = det_specific('S2', 'angular_sep2', 'F', delta_, 'baseline')

In [ ]:
plot_angsep(yieldM, 'S2_HZangsep_M')
plot_angsep(yieldK, 'S2_HZangsep_K')
plot_angsep(yieldG, 'S2_HZangsep_G')
plot_angsep(yieldF, 'S2_HZangsep_F')